# Infer NUCLEI - part 1

--------------

## OBJECTIVE:  Infer sub-cellular component #1: NUCLEI  in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components Nuclei (NU).  

Dependencies:
SOMA and CYTOSOL inference rely on the Nuclei inference.  Therefore all of the sub-cellular objects rely on the NU segmentation.


In [ ]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from re import S

import numpy as np

import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation

from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO

from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice

from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 

from skimage import filters
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label


# # package for io 
# from aicsimageio import AICSImage



from napari.utils.notebook_display import nbscreenshot


#import .infer_subc.base

from infer_subc.base import (etree_to_dict,
                            get_raw_meta_data,
                            median_filter_slice_by_slice,
                            read_input_image,
                            simple_intensity_normalization,
                            export_ome_tiff,
                            threshold_li_log,
                            threshold_otsu_log,
                            log_transform,
                            read_input_image
                            )


# OBJECTIVE :  infer NUCLEI
 
## summary of steps


INPUT
- channel 0


PRE-PROCESSING
-  scale to min 0, max 1.0
- median Filter window 10
  CORE-PROCESSING
  - threshold method minimum cross-entropy.  
    - objects 50-400 pixels, 
    - threshold smoothing scale: 1 pixel
    - threshold correction factor: 1.2 (more stringent)
    - lower / upper bounds  (.1,1)
    - log transformed thresholding
    - fill holes


POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- labels of NUCLEI


> #### Note:  in later steps we will limit each analysis to a single object, but at this stage we have multiple



NOTE:  using Allen Cell Segmenter  [Nucleophosmin](https://www.allencell.org/cell-observations/category/nucleophosmin) might e a good place to start.  [playground_npm1.ipynb](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_npm1.ipynb) and [npm1.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1.py) and [npm1_SR.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1_SR.py) 

In [1]:
# Get and load Image for processing
For testing purposes... TODO: build a nice wrapper for this.


SyntaxError: invalid syntax (474537597.py, line 2)

In [3]:
# assumptions:   .czi "unmixed" collections.
czi_img_folder = f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data"
list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[5])
#Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.

test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

img_data.shape
channel_names = meta_dict['name']

img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

chan_name = 'nuclei'
out_path = './'
img_name = 'NU_object'
data_in = img_data.copy()[[0],:,:,:]
meta_in = meta_dict 

chan_names = meta_in['name']

viewer = napari.view_image(
    img_data,
    channel_axis=0,
    name=chan_names,
    scale=scale
)
viewer.scale_bar.visible = True


/Users/ahenrie/Projects/Imaging/mcz_subcell/data/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


NameError: name 'read_input_image' is not defined


# IMAGE PROCESSING Objective 1:  infer NUCLEI
 
 [OUTLINE: Objective #1](#summary-of-objectives)
## summary of steps


INPUT
- channel 0


PRE-PROCESSING
-  scale to min 0, max 1.0
- median Filter window 10
  CORE-PROCESSING
  - threshold method minimum cross-entropy.  
    - objects 50-400 pixels, 
    - threshold smoothing scale: 1 pixel
    - threshold correction factor: 1.2 (more stringent)
    - lower / upper bounds  (.1,1)
    - log transformed thresholding
    - fill holes


POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- labels of NUCLEI


> #### Note:  in later steps we will limit each analysis to a single object, but at this stage we have multiple



NOTE:  using Allen Cell Segmenter  [Nucleophosmin](https://www.allencell.org/cell-observations/category/nucleophosmin) might e a good place to start.  [playground_npm1.ipynb](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_npm1.ipynb) and [npm1.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1.py) and [npm1_SR.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1_SR.py) 